In [15]:
import math
import numpy as np
import matplotlib.pyplot as plt

class Option(object):
    def __init__(self, S0, K, r, T, N, params):
        self.S0 = S0 
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N) 
        self.STs = None  # Stock Price Tree

        # These parameters will be inherited to BinomialPricing and trinomialpricing codes
        self.pu = params.get("pu", 0)  # Up state probability 
        self.pd = params.get("pd", 0)  # Down state prob
        self.div = params.get("div", 0)  # Dividend yield
        self.sigma = params.get("sigma", 0)  # Volatility
        self.is_call = params.get("is_call", True)  # Type of option(call or put)
        self.is_european = params.get("is_eu", True)  # Type of option(European or American)

        self.dt = T/float(N)  # time steps
        self.df = math.exp(-(r-self.div) * self.dt)  # Discount factor


In [17]:
import math
import numpy as np
import matplotlib.pyplot as plt
# Ensure Option is already defined
# from option import Option  # Uncomment if running as a separate script

class BinomialTree(Option):

    def _setup_parameters_(self):
        self.u = 1 + self.pu  
        self.d = 1 - self.pd  
        self.qu = (math.exp((self.r-self.div)*self.dt) -
                   self.d)/(self.u-self.d)
        self.qd = 1-self.qu

    def _initialize_stock_price_tree_(self):
        self.STs = [np.array([self.S0])]
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches*self.u,
                                 [prev_branches[-1]*self.d]))
            self.STs.append(st)  

    def _initialize_payoffs_tree_(self):
        return np.maximum(
            0, (self.STs[self.N]-self.K) if self.is_call
            else (self.K-self.STs[self.N]))

    def __check_early_exercise__(self, payoffs, node):
        early_ex_payoff = (self.STs[node] - self.K) if self.is_call else (self.K - self.STs[node])
        return np.maximum(payoffs, early_ex_payoff)

    def _traverse_tree_(self, payoffs):
        for i in reversed(range(self.N)):
            payoffs = (payoffs[:-1] * self.qu +
                      payoffs[1:] * self.qd) * self.df
            if not self.is_european:
                payoffs = self.__check_early_exercise__(payoffs, i)
        return payoffs

    def __begin_tree_traversal__(self):
        payoffs = self._initialize_payoffs_tree_()
        return self._traverse_tree_(payoffs)

    def price(self):
        self._setup_parameters_()
        self._initialize_stock_price_tree_()
        payoffs = self.__begin_tree_traversal__()
        return payoffs[0]

# Example usage
if __name__ == "__main__":
    am_option = BinomialTree(100, 80, 0.06, 0.5, 2, {"pu": 0.2, "pd": 0.2, "is_call": False, "is_eu": False})
    print("Price of American option:", am_option.price())


Price of American option: 3.3172913262501407


In [18]:
import math
import numpy as np
# Ensure BinomialTree is already defined
# from binomialpricing import BinomialTree  # Uncomment if running as a separate script

class TrinomialTree(BinomialTree):

    def _setup_parameters_(self):
        self.u = math.exp(self.sigma*math.sqrt(2.*self.dt))
        self.d = 1/self.u
        self.m = 1
        self.qu = ((math.exp((self.r-self.div) * self.dt/2.) -
                    math.exp(-self.sigma * math.sqrt(self.dt/2.))) /
                   (math.exp(self.sigma * math.sqrt(self.dt/2.)) -
                    math.exp(-self.sigma * math.sqrt(self.dt/2.))))**2
        self.qd = ((math.exp(self.sigma * math.sqrt(self.dt/2.)) -
                    math.exp((self.r-self.div) * self.dt/2.)) /
                   (math.exp(self.sigma * math.sqrt(self.dt/2.)) -
                    math.exp(-self.sigma * math.sqrt(self.dt/2.))))**2
        self.qm = 1 - self.qu - self.qd

    def _initialize_stock_price_tree_(self):
        self.STs = [np.array([self.S0])]
        for i in range(self.N):
            prev_nodes = self.STs[-1]
            self.ST = np.concatenate(
                (prev_nodes*self.u, [prev_nodes[-1]*self.m,
                                     prev_nodes[-1]*self.d]))
            self.STs.append(self.ST)

    def _traverse_tree_(self, payoffs):
        for i in reversed(range(self.N)):
            payoffs = (payoffs[:-2] * self.qu +
                       payoffs[1:-1] * self.qm +
                       payoffs[2:] * self.qd) * self.df
            if not self.is_european:
                payoffs = self.__check_early_exercise__(payoffs, i)
        return payoffs

# Example usage
if __name__ == "__main__":
    print("Price of European put:", TrinomialTree(100, 80, 0.06, 0.5, 2, {"sigma": 0.3, "is_call": False}).price())
    print("Price of American put:", TrinomialTree(100, 80, 0.06, 0.5, 2, {"sigma": 0.3, "is_call": False, "is_eu": False}).price())


Price of European put: 0.8225260069146118
Price of American put: 0.8225260069146118


In [19]:
# Test Binomial Tree Model
am_option_binomial = BinomialTree(100, 80, 0.06, 0.5, 2, {"pu": 0.2, "pd": 0.2, "is_call": False, "is_eu": False})
print("Price of American option using Binomial Tree:", am_option_binomial.price())

# Test Trinomial Tree Model
eu_put_trinomial = TrinomialTree(100, 80, 0.06, 0.5, 2, {"sigma": 0.3, "is_call": False})
am_put_trinomial = TrinomialTree(100, 80, 0.06, 0.5, 2, {"sigma": 0.3, "is_call": False, "is_eu": False})

print("Price of European put using Trinomial Tree:", eu_put_trinomial.price())
print("Price of American put using Trinomial Tree:", am_put_trinomial.price())

Price of American option using Binomial Tree: 3.3172913262501407
Price of European put using Trinomial Tree: 0.8225260069146118
Price of American put using Trinomial Tree: 0.8225260069146118
